# Ensemble Methods - Boosting

## Objectives

- Describe boosting algorithms
- Implement boosting models with `sklearn` 
- Implement boosting models with `XGBoost`

# Intro

One of the problems with using single decision trees and random forests is that, once I make a split, I can't go back and consider how another feature varies across the whole dataset. But suppose I were to consider **my tree's errors**. The fundamental idea of ***boosting*** is to start with a **weak learner** and then to use information about its errors to build a new model that can supplement the original model.

Though the individual learners are weak, the idea is to train iteratively in order to produce a better predictor. More specifically, the first learner will be trained on the data as it stands, but future learners will be trained on modified versions of the data. The point of the modifications is to highlight the "hard-to-predict-accurately" portions of the data.

## Enter: Boosting 

Boosting is a sequential process, where each subsequent model attempts to correct the errors of the previous model. The succeeding models are dependent on the previous model. Let’s understand the way boosting works in the below steps.

1. A subset is created from the original dataset.
2. Initially, all data points are given equal weights.
3. A base model is created on this subset.
4. This model is used to make predictions on the whole dataset.
![](https://cdn.analyticsvidhya.com/wp-content/uploads/2015/11/dd1-e1526989432375.png)


5. Errors are calculated using the actual values and predicted values.
6. The observations which are incorrectly predicted, are given higher weights (here, the three misclassified blue-plus points will be given higher weights)
7. Another model is created and predictions are made on the dataset (this model tries to correct the errors from the previous model)
![](https://cdn.analyticsvidhya.com/wp-content/uploads/2015/11/dd2-e1526989487878.png)


8. Similarly, multiple models are created, each correcting the errors of the previous model.
9. The final model (strong learner) is the weighted mean of all the models (weak learners).
![](https://www.analyticsvidhya.com/wp-content/uploads/2015/11/boosting10-300x205.png)



Thus, the boosting algorithm combines a number of weak learners to form a strong learner. The individual models would not perform well on the entire dataset, but they work well for some part of the dataset. Thus, each model actually boosts the performance of the ensemble.
![](https://cdn.analyticsvidhya.com/wp-content/uploads/2015/11/dd4-e1526551014644.png)

## Set Up

Scenario: predict whether a car was made in US

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import precision_score, recall_score, confusion_matrix

from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv('data/cars.csv', na_values = ' ')
df['target'] = df[' brand'] == ' US.'
X = df.drop(['target', ' brand'], axis=1)
y = df['target']

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)

In [4]:
X_train.head()

,mpg,cylinders,cubicinches,hp,weightlbs,time-to-60,year
230,15.0,8,383.0,170,3563.0,10,1971
75,38.0,6,262.0,85,3015.0,17,1983
248,28.0,4,97.0,75,2155.0,16,1977
243,25.0,4,110.0,87,2672.0,18,1971
101,12.0,8,400.0,167,4906.0,13,1974


In [5]:
si = SimpleImputer()

si.fit(X_train)

X_tr_im = si.transform(X_train)
X_te_im = si.transform(X_test)

### Baseline Understanding

In [6]:
# Modelless baseline
y_train.value_counts(normalize=True)

True     0.666667
False    0.333333
Name: target, dtype: float64

In [7]:
# Decision tree with max_depth = 5
dt_baseline = DecisionTreeClassifier(max_depth=5)

dt_baseline.fit(X_tr_im, y_train)

print(f"Train Score: {dt_baseline.score(X_tr_im, y_train)}")
print(f"Test Score: {dt_baseline.score(X_te_im, y_test)}")

Train Score: 0.958974358974359
Test Score: 0.8333333333333334


## AdaBoost (Adaptive Boosting)

**AdaBoost** works by iteratively adapting two related series of weights, one attached to the datapoints and the other attached to the learners themselves. Datapoints that are incorrectly classified receive greater weights for the next learner in the sequence. That way, future learners will be more likely to focus on those datapoints. At the end of the sequence, the learners that make better predictions, especially on the datapoints that are more resistant to correct classification, receive more weight in the final "vote" that determines the ensemble's prediction.


### The Steps:

1. Initially, all observations in the dataset are given equal weights.
2. A model is built on a subset of data.
3. Using this model, predictions are made on the whole dataset.
4. Errors are calculated by comparing the predictions and actual values.
5. While creating the next model, higher weights are given to the data points which were predicted incorrectly.
6. Weights can be determined using the error value. For instance, higher the error more is the weight assigned to the observation.
7. This process is repeated until the error function does not change, or the maximum limit of the number of estimators is reached.


## AdaBoost in Scikit-Learn

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html

In [8]:
# Import!
from sklearn.ensemble import AdaBoostClassifier

In [15]:
# Instantiate and fit, after exploring hyperparameters
ada = AdaBoostClassifier(random_state=42, learning_rate=.1, n_estimators=100)

ada.fit(X_tr_im, y_train)

AdaBoostClassifier(learning_rate=0.1, n_estimators=100, random_state=42)

In [16]:
print(f"Train Score: {ada.score(X_tr_im, y_train)}")
print(f"Test Score: {ada.score(X_te_im, y_test)}")

Train Score: 0.9538461538461539
Test Score: 0.9242424242424242


#### Hyperparameter Tuning

Let's see if we can do better by trying different hyperparameter values:

In [17]:
# Don't really need to do this, but nice to start with a clean slate
ada = AdaBoostClassifier(random_state = 123)

# Let's define a param grid together!
param_grid = {
    'learning_rate': [.01, .1, .5, 1],
    'n_estimators': [10, 50, 100, 1000]
}

# Create our grid search
gs = GridSearchCV(estimator=ada, param_grid=param_grid, cv = 5)

# Fit our grid search
gs.fit(X_tr_im, y_train)

GridSearchCV(cv=5, estimator=AdaBoostClassifier(random_state=123),
             param_grid={'learning_rate': [0.01, 0.1, 0.5, 1],
                         'n_estimators': [10, 50, 100, 1000]})

In [19]:
print(f"Train Score: {gs.best_estimator_.score(X_tr_im, y_train)}")
print(f"Test Score: {gs.best_estimator_.score(X_te_im, y_test)}")

Train Score: 0.9435897435897436
Test Score: 0.9242424242424242


In [20]:
# Check out the best parameters found by the search
gs.best_params_

{'learning_rate': 0.01, 'n_estimators': 1000}

# Gradient Boosting

> Use gradient descent to improve the model

![](images/gradient_boosting_residuals.png)

**Gradient Boosting** works by training each new learner on the residuals of the model built with the learners that have so far been constructed. That is, Model $n+1$ (with $n+1$ learners) will focus on the predictions of Model $n$ (with only $n$ learners) that were **most off the mark**. As the training process repeats, the learners learn and the residuals get smaller. I would get a sequence going:

Model 0 is very simple. Perhaps it merely predicts the mean: $\hat{y}_0 = \bar{y}$;

Model 1's predictions would then be the sum of (i) Model 0's predictions and (ii) the predictions of the model fitted to Model 0's residuals: $\hat{y}_1 = \hat{y}_0 + \hat{(y - \hat{y})}_{err0}$; 

Now iterate: Model 2's predictions will be the sum of (i) Model 0's predictions, (ii) the predictions of the model fitted to Model 0's residuals, and (iii) the predictions of the model fitted to Model 1's residuals: $\hat{y}_2 = \hat{y}_0 + \hat{(y - \hat{y})}_{err0} + \hat{(y - \hat{y})}_{err1}$

Et cetera, et cetera!

$\rightarrow$ How does gradient boosting work for a classification problem? How do we even make sense of the notion of a gradient in that context? The short answer is that we appeal to the probabilities associated with the predictions for the various classes. See more on this topic [here](https://sefiks.com/2018/10/29/a-step-by-step-gradient-boosting-example-for-classification/). <br/> $\rightarrow$ Why is this called "_gradient_ boosting"? Because using a model's residuals to build a new model is using information about the derivative of that model's loss function. See more on this topic [here](https://www.ritchievink.com/blog/2018/11/19/algorithm-breakdown-why-do-we-call-it-gradient-boosting/).

### SKLearn's Gradient Boosting

In [21]:
# Import! 
from sklearn.ensemble import GradientBoostingClassifier

In [22]:
# Instantiate and fit - let's use just a few estimators to start
gb_sklearn = GradientBoostingClassifier(max_depth=2,
                                        n_estimators=10,
                                        random_state=81)

gb_sklearn.fit(X_tr_im, y_train)

GradientBoostingClassifier(max_depth=2, n_estimators=10, random_state=81)

In [23]:
print(f"Train Score: {gb_sklearn.score(X_tr_im, y_train)}")
print(f"Test Score: {gb_sklearn.score(X_te_im, y_test)}")

Train Score: 0.9025641025641026
Test Score: 0.8484848484848485


## Comparing gradient boosting with many estimators 

In [24]:
# More estimators
gb_more = GradientBoostingClassifier(max_depth=2,
                                     n_estimators=100,
                                     random_state=123)
gb_more.fit(X_tr_im, y_train)

# Even more estimators
gb_evenmore = GradientBoostingClassifier(max_depth=2,
                                         n_estimators=1000,
                                         random_state=123)
gb_evenmore.fit(X_tr_im, y_train)

GradientBoostingClassifier(max_depth=2, n_estimators=1000, random_state=123)

In [25]:
print(f"Train Score: {gb_more.score(X_tr_im, y_train)}")
print(f"Test Score: {gb_more.score(X_te_im, y_test)}")

Train Score: 0.9846153846153847
Test Score: 0.9242424242424242


In [26]:
print(f"Train Score: {gb_evenmore.score(X_tr_im, y_train)}")
print(f"Test Score: {gb_evenmore.score(X_te_im, y_test)}")

Train Score: 1.0
Test Score: 0.8636363636363636


In [31]:
gb_next = GradientBoostingClassifier(max_depth=1,
                                     n_estimators=200,
                                     random_state=123)

gb_next.fit(X_tr_im, y_train)
print(f"Train Score: {gb_next.score(X_tr_im, y_train)}")
print(f"Test Score: {gb_next.score(X_te_im, y_test)}")

Train Score: 0.958974358974359
Test Score: 0.9242424242424242


# XGBoost

From [XGBoost's documentation](https://xgboost.readthedocs.io/):

>_**XGBoost** is an optimized distributed gradient boosting library designed to be highly **efficient**, **flexible** and **portable**. It implements machine learning algorithms under the Gradient Boosting framework. XGBoost provides a parallel tree boosting (also known as GBDT, GBM) that solve many data science problems in a fast and accurate way. The same code runs on major distributed environment (Hadoop, SGE, MPI) and can solve problems beyond billions of examples._

XGBoost (eXtreme Gradient Boosting) is an advanced implementation of the gradient boosting algorithm. XGBoost has proved to be a highly effective ML algorithm, extensively used in machine learning competitions and hackathons. XGBoost has high predictive power and is almost 10 times faster than the other gradient boosting techniques. It also includes a variety of regularization which reduces overfitting and improves overall performance. Hence it is also known as ‘regularized boosting‘ technique.

**Pros of XGBoost:** 

1. Regularization:
    - Standard GBM implementation has no regularisation like XGBoost.
    - Thus XGBoost also helps to reduce overfitting.
    
    
2. Parallel Processing:
    - XGBoost implements parallel processing and is faster than GBM .
    - XGBoost also supports implementation on Hadoop.
    
    
3. High Flexibility:
    - XGBoost allows users to define custom optimization objectives and evaluation criteria adding a whole new dimension to the model.
    
    
4. Handling Missing Values:
    - XGBoost has an in-built routine to handle missing values.
    
    
5. Tree Pruning:
    * XGBoost makes splits up to the max_depth specified and then starts pruning the tree backwards and removes splits beyond which there is no positive gain.
    
    
6. Built-in Cross-Validation:
    * XGBoost allows a user to run a cross-validation at each iteration of the boosting process and thus it is easy to get the exact optimum number of boosting iterations in a single run.

Note - XGBoost is not in SKLearn!

[Documentation for the Classifier](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBClassifier)

In [32]:
# Import!
import xgboost as xgb

In [40]:
# Instantiate and fit
xgb_model = xgb.XGBClassifier(random_state=7, 
                              learning_rate=.1, 
                              max_depth=2, 
                              base_score=.66)

xgb_model.fit(X_tr_im, y_train)

XGBClassifier(base_score=0.66, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=7,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [41]:
print(f"Train Score: {xgb_model.score(X_tr_im, y_train)}")
print(f"Test Score: {xgb_model.score(X_te_im, y_test)}")

Train Score: 0.9743589743589743
Test Score: 0.9090909090909091


## Discuss: Bagging vs Boosting?

How are they similar?

- they're both ensemble techniques


How are they different?

- bagging happens in parallel, boosting happens sequentially
- bagging focuses on subsetting the data (rows and features for random forest) to prevent overfitting, while boosting focuses on fixing the mistakes of the previous weak model


## Resources

- [Great short podcast on Ensemble Methods](http://lineardigressions.com/episodes/2017/1/22/ensemble-algorithms)
- [Slideshow on bagging and boosting ensemble methods](http://www2.stat.duke.edu/~rcs46/lectures_2017/08-trees/08-tree-advanced.pdf)
- [How to explain Gradient Boosting](https://explained.ai/gradient-boosting/index.html)
- [Thorough post on AdaBoost from Data Camp](https://www.datacamp.com/community/tutorials/adaboost-classifier-python)
- [Complete Guide to AdaBoost for Beginners from Analytics Vidhya](https://www.analyticsvidhya.com/blog/2021/09/adaboost-algorithm-a-complete-guide-for-beginners/)


-----

# Level Up: Move to Regression!

In [42]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor

### Galaxy Data

In [43]:
galaxies = pd.read_csv('data/COMBO17.csv')
galaxies.head()

,Nr,Rmag,e.Rmag,ApDRmag,mumax,Mcz,e.Mcz,MCzml,chi2red,UjMAG,...,UFS,e.UFS,BFS,e.BFS,VFD,e.VFD,RFS,e.RFS,IFD,e.IFD
0,6,24.995,0.097,0.935,24.214,0.832,0.036,1.400,0.64,-17.67,...,0.01870,0.00239,0.01630,0.00129,0.017300,0.00141,0.01650,0.000434,0.02470,0.00483
1,9,25.013,0.181,-0.135,25.303,0.927,0.122,0.864,0.41,-18.28,...,0.00706,0.00238,0.00420,0.00115,0.003930,0.00182,0.00723,0.000500,0.00973,0.00460
2,16,24.246,0.054,0.821,23.511,1.202,0.037,1.217,0.92,-19.75,...,0.01260,0.00184,0.01830,0.00115,0.018800,0.00167,0.02880,0.000655,0.05700,0.00465
3,21,25.203,0.128,0.639,24.948,0.912,0.177,0.776,0.39,-17.83,...,0.01410,0.00186,0.01180,0.00110,0.009670,0.00204,0.01050,0.000416,0.01340,0.00330
4,26,25.504,0.112,-1.588,24.934,0.848,0.067,1.330,1.45,-17.69,...,0.00514,0.00170,0.00102,0.00127,0.000039,0.00160,0.00139,0.000499,0.00590,0.00444


This is a dataset about galaxies. The Mcz and MCzml columns are measures of redshift, which is our target. Mcz is usually understood to be a better measure, so that will be our target column. Many of the other columns have to do with various measures of galaxies' magnitudes. For more on the dataset, see [here](https://astrostatistics.psu.edu/datasets/COMBO17.html).

In [44]:
galaxies.columns

Index(['Nr', 'Rmag', 'e.Rmag', 'ApDRmag', 'mumax', 'Mcz', 'e.Mcz', 'MCzml',
       'chi2red', 'UjMAG', 'e.UjMAG', 'BjMAG', 'e.BjMAG', 'VjMAG', 'e.VjMAG',
       'usMAG', 'e.usMAG', 'gsMAG', 'e.gsMAG', 'rsMAG', 'e.rsMAG', 'UbMAG',
       'e.UbMAG', 'BbMAG', 'e.BbMAG', 'VnMAG', 'e.VbMAG', 'S280MAG',
       'e.S280MA', 'W420FE', 'e.W420FE', 'W462FE', 'e.W462FE', 'W485FD',
       'e.W485FD', 'W518FE', 'e.W518FE', 'W571FS', 'e.W571FS', 'W604FE',
       'e.W604FE', 'W646FD', 'e.W646FD', 'W696FE', 'e.W696FE', 'W753FE',
       'e.W753FE', 'W815FS', 'e.W815FS', 'W856FD', 'e.W856FD', 'W914FD',
       'e.W914FD', 'W914FE', 'e.W914FE', 'UFS', 'e.UFS', 'BFS', 'e.BFS', 'VFD',
       'e.VFD', 'RFS', 'e.RFS', 'IFD', 'e.IFD'],
      dtype='object')

In [45]:
galaxies.isnull().sum().sum()

50

In [46]:
galaxies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3462 entries, 0 to 3461
Data columns (total 65 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Nr        3462 non-null   int64  
 1   Rmag      3462 non-null   float64
 2   e.Rmag    3462 non-null   float64
 3   ApDRmag   3462 non-null   float64
 4   mumax     3462 non-null   float64
 5   Mcz       3462 non-null   float64
 6   e.Mcz     3462 non-null   float64
 7   MCzml     3462 non-null   float64
 8   chi2red   3462 non-null   float64
 9   UjMAG     3462 non-null   float64
 10  e.UjMAG   3462 non-null   float64
 11  BjMAG     3462 non-null   float64
 12  e.BjMAG   3462 non-null   float64
 13  VjMAG     3462 non-null   float64
 14  e.VjMAG   3462 non-null   float64
 15  usMAG     3462 non-null   float64
 16  e.usMAG   3462 non-null   float64
 17  gsMAG     3462 non-null   float64
 18  e.gsMAG   3462 non-null   float64
 19  rsMAG     3462 non-null   float64
 20  e.rsMAG   3462 non-null   floa

In [47]:
galaxies = galaxies.dropna()

Let's collect together the columns that have high correlation with Mcz, our target:

In [48]:
preds = []
for ind in galaxies.corr()['Mcz'].index:
    if abs(galaxies.corr()['Mcz'][ind]) > 0.5:
        preds.append(ind)

In [49]:
galaxies[preds].corr()

,Mcz,e.Mcz,MCzml,UjMAG,BjMAG,VjMAG,usMAG,gsMAG,rsMAG,UbMAG,BbMAG,VnMAG,S280MAG
Mcz,1.000000,0.613071,0.872016,-0.699550,-0.628226,-0.641983,-0.698304,-0.652170,-0.631398,-0.699731,-0.659506,-0.641870,-0.700453
e.Mcz,0.613071,1.000000,0.590179,-0.170296,-0.122309,-0.122616,-0.168326,-0.126491,-0.114022,-0.170208,-0.130680,-0.122546,-0.169344
MCzml,0.872016,0.590179,1.000000,-0.565663,-0.520006,-0.532880,-0.563591,-0.536750,-0.525495,-0.565406,-0.544325,-0.532837,-0.562474
UjMAG,-0.699550,-0.170296,-0.565663,1.000000,0.933252,0.961088,0.999865,0.970124,0.954853,0.999986,0.975476,0.960981,0.963415
BjMAG,-0.628226,-0.122309,-0.520006,0.933252,1.000000,0.950728,0.932328,0.954006,0.947597,0.932831,0.957836,0.950694,0.885382
VjMAG,-0.641983,-0.122616,-0.532880,0.961088,0.950728,1.000000,0.959432,0.997956,0.999324,0.960265,0.992178,0.999997,0.894171
usMAG,-0.698304,-0.168326,-0.563591,0.999865,0.932328,0.959432,1.000000,0.968971,0.953027,0.999903,0.974515,0.959320,0.964972
gsMAG,-0.652170,-0.126491,-0.536750,0.970124,0.954006,0.997956,0.968971,1.000000,0.995436,0.969562,0.995887,0.997923,0.914118
rsMAG,-0.631398,-0.114022,-0.525495,0.954853,0.947597,0.999324,0.953027,0.995436,1.000000,0.953909,0.988695,0.999341,0.882782
UbMAG,-0.699731,-0.170208,-0.565406,0.999986,0.932831,0.960265,0.999903,0.969562,0.953909,1.000000,0.975035,0.960155,0.964189


These various magnitude columns all have high correlations **with one another**! Let's try a simple model with just the S280MAG column, since it has the highest correlation with Mcz.

In [50]:
galaxies_x = galaxies['S280MAG']
galaxies_y = galaxies['Mcz']

Since we only have one predictor, we can visualize the correlation with the target! We can also reshape it for modeling purposes!

In [51]:
galaxies_x_rev = galaxies_x.values.reshape(-1, 1)

In [52]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(galaxies_x_rev, galaxies_y)

## AdaBoost Regression

In [53]:
abr = AdaBoostRegressor(random_state=42)

abr.fit(x_train2, y_train2)

AdaBoostRegressor(random_state=42)

In [54]:
abr.score(x_test2, y_test2)

0.5288180063130403

## Gradient Boosted Regression

In [55]:
gbr = GradientBoostingRegressor(random_state=42)

gbr.fit(x_train2, y_train2)

GradientBoostingRegressor(random_state=42)

In [56]:
gbr.score(x_test2, y_test2)

0.5500523652255273

## XGBoost Regression

In [57]:
xgbr = xgb.XGBRegressor(random_state=42)

xgbr.fit(x_train2, y_train2)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [58]:
xgbr.score(x_test2, y_test2)

0.4794495285396214